![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png)  ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
# Contando palabras: Construye una aplicacion que cuente palabras de forma eficiente

Este laboratorio usara las tecnologias descritas en los materiales del curso sobre Spark para desarrollar una aplicacion de conteo de palabras. 

Con el uso masivo de Internet y las redes sociales, el volumen de texto no estructurado esta creciendo dramaticamente, y Spark es una gran herramienta para analizar este tipo de datos. En esta PEC, vamos a escribir codigo para encontrar las palabras mas comunes en un texto generado en latin, el ya conocido [Lorem Ipsum](https://www.lipsum.com/).


Lo mas interesante de la forma de trabajar en esta practica es que podria escalarse para, por ejemplo, encontrar las palabras mas comunes en Wikipedia.

## Durante esta PEC vamos a cubrir:

* *Parte 1:* Creación de un RDD y un pair RDD
* *Parte 2:* Contar palabras usando un pair RDD
* *Parte 3:* Encontrar las palabras individuales y su frecuencia de aparicion media
* *Parte 4:* Aplicar las funcionalidades desarrolladas a un archivo de texto* 
* *Parte 5:* Calcular algunos estadisticos*


> Como referencia a todos los detalles de los metodos que se usan en esta practica usar:
> * [API Python de Spark](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD)

## Parte 1: Creacion de un RDD y un pair RDDs

En esta seccion, exploraremos como crear RRDs usando `parallelize` y como aplicar pair RDDs al problema del conteo de palabras.

### (0) Configuración del entorno python + spark

In [1]:
#import findspark
# findspark.init()
import pyspark
import random
sc = pyspark.SparkContext(master="local", appName="Gonzalezjulvez")

Exception: Java gateway process exited before sending its port number

### (1a) Creación de un RDD
Empezemos generando un RDD a partir de una lista de Python y el metodo `sc.parallelize`. Luego mostraremos por pantalla el tipo de la variable generada.

In [2]:
wordsList = ['cat', 'elephant', 'rat', 'rat', 'cat']
wordsRDD = sc.parallelize(wordsList, 4)
# Print out the type of wordsRDD
type(wordsRDD)

pyspark.rdd.RDD

### (1b) Crear el plural de las palabras y testear

Vamos a utilizar una transformacion `map()` para incorporar la letra 's' a cada uno de los strings almacenados en el RDD que acabamos de crear. Vamos a definir una funcion de Python que devuelva una palabra, que se le ha pasado como parametro, incorporando una "s" al final de la misma. Reemplazar el texto `<FILL IN>` con la solucion propuesta. Despues de haber definido correctamente la funcion `makePlural`, ejecutar la segunda celda que contiene un assert de test. Si la solucion es correcta, se imprimira `1 test passed`.

Esta sera la forma habitual de trabajar en las PECs. Los ejercicios contendran una explicacion de lo que se espera, seguido de una celda de codigo con uno o mas `<FILL IN>`. Las celdas que necesiten ser modificadas contendran el texto `# TODO: Replace <FILL IN> with appropriate code` en la primera linea.

Una vez se hayan sustituido todos los `<FILL IN>` por el codigo Python adecuado, ejecutar la celda, y posteriormente ejecutar la celda siguiente de test para comprobar que que la solucion es la esperada.

In [3]:
# TODO: Replace <FILL IN> with appropriate code
def makePlural(word):
    """Adds an 's' to `word`.

    Note:
        This is a simple function that only adds an 's'.  

    Args:
        word (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    
    return word + 's'

makePlural('cat')

'cats'

In [4]:
# TEST Pluralize and test (1b)
assert makePlural('rat') == 'rats', 'incorrect result: makePlural does not add an s' 

### (1c) Aplicar `makePlural` a nuestro RDD

Ahora es el momento de aplicar nuestra funcion `makePlural()` a todos los elementos del RDD usando una transformacion [map()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map). Posteriormente ejecutar la accion [collect()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.collect) para obtener el RDD transformado.

In [5]:
# TODO: Replace <FILL IN> with appropriate code
pluralRDD = wordsRDD.map(makePlural)
pluralRDD.collect()

['cats', 'elephants', 'rats', 'rats', 'cats']

In [6]:
# TEST Apply makePlural to the base RDD(1c)
assert pluralRDD.collect() == ['cats', 'elephants', 'rats', 'rats', 'cats'], 'incorrect values for pluralRDD'

### (1d) Ejecutar una funcion `lambda` en un `map`

Vamos a crear el mismo RDD usando una `lambda` function en lugar de una funcion con nombre.

In [7]:
# TODO: Replace <FILL IN> with appropriate code
pluralLambdaRDD = wordsRDD.map(lambda word : makePlural(word))
print(pluralLambdaRDD.collect())

['cats', 'elephants', 'rats', 'rats', 'cats']


In [8]:
# TEST Pass a lambda function to map (1d)
assert pluralLambdaRDD.collect() == ['cats', 'elephants', 'rats', 'rats', 'cats'], 'incorrect values for pluralLambdaRDD (1d)'

### (1e) Numero de caracteres de cada una de las palabras

Ahora vamos a usar un `map()` y una funcion lambda `lambda` para obtener el numero de caracteres de cada palabra. Usaremos `collect` para guardar este resultado directamente en una variable.

In [9]:
# TODO: Replace <FILL IN> with appropriate code
pluralLengths = (pluralRDD.map(lambda word : len(word)).collect())
print(pluralLengths)

[4, 9, 4, 4, 4]


In [10]:
# TEST Length of each word (1e)
assert pluralLengths == [4, 9, 4, 4, 4], 'incorrect values for pluralLengths'

### (1f) Pair RDDs

El siguiente paso para completar nuestro programa de conteo de palabras en crear un nuevo tipo de RDD, llamado pair RDD. Un pair RDD es un RDD donde cada elemento es un tupla del estilo `(k, v)` donde `k` es la clave y `v` es su valor correspondiente. En este ejemplo, crearemos una pair RDD consistente en tuplas con el formato `('<word>', 1)` para cada elemento de nuestro RDD basico.

Podemos crear nuestro pair RDD usando una transformacion `map()` con una `lambda()` function que cree un nuevo RDD.

In [11]:
# TODO: Replace <FILL IN> with appropriate code
wordPairs = wordsRDD.map(lambda word : (word,1))
print(wordPairs.collect())

[('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)]


In [12]:
# TEST Pair RDDs (1f)
assert wordPairs.collect() == [('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)], 'incorrect value for wordPairs'

## Parte 2: Contar palabras usando un pair RDD

Ahora, contaremos el numero de veces que una palabra en particular aparece en el RDD. Esta operacion se puede realizar de una infinidad de maneras, pero algunas seran mucho menos eficientes que otras.

Un solucion muy sencilla seria usar `collect()` sobre todos los elementos devolverlos al driver y alli contarlos. Mientras esta forma de trabajar podria funcionar con textos relativamente cortos, nosotros lo que queremos es poder trabajar con textos de cualquier longitud. Adicionalmente, ejecutar todo el calculo en el driver es mucho mas lento que ejecutarlo en paralelo en los workers. Por estos motivos, en esta practica usaremos operaciones paralelizables.

%md
### (2a) Usando `groupByKey()`
Una primera solucion a nuestro problema, luego veremos que hay otras mucho mas eficientes, se podria basar en la transformacion [groupByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey). Como su nombre indica, la transformacion `groupByKey()` agrupa todos los elementos de un RDD que compartan la misma clave en una unica lista dentro de una de las particiones.

Esta operacion plantea dos problemas:
  + Esta operacion necesita mover todos los valores dentro de la particion adecuada. Esto satura la red. 
  + Las listas generadas pueden llegar a ser muy grandes llegando incluso a saturar la memoria de alguno de los trabajadadores
  
Utiliza `groupByKey()` para generar un pair RDD del tipo `('word', iterator)`.

In [13]:
# TODO: Replace <FILL IN> with appropriate code
# Note that groupByKey requires no parameters
wordsGrouped = wordPairs.groupByKey()
for key, value in wordsGrouped.collect():
    print('{0}: {1}'.format(key, list(value)))

cat: [1, 1]
elephant: [1]
rat: [1, 1]


In [14]:
# TEST groupByKey() approach (2a)
assert sorted(wordsGrouped.mapValues(lambda x: list(x)).collect()) == [('cat', [1, 1]), ('elephant', [1]), ('rat', [1, 1])], 'incorrect value for wordsGrouped'

### (2b) Utiliza `groupByKey()` para obtener los conteos

Usando la transformacion `groupByKey()` crea un RDD que contenga 2 elementos, donde cada uno de ellos sea un par palabra (clave) iterador de Python (valor).

Luego suma todos los valores de iterador usando una transformacion `map()`. El resultado debe ser un pair RDD que contenga las parejas (word, count).

In [15]:
# TODO: Replace <FILL IN> with appropriate code
wordCountsGrouped = wordsGrouped.map(lambda x : (x[0], len(x[1])))
print(wordCountsGrouped.collect())

[('cat', 2), ('elephant', 1), ('rat', 2)]


In [16]:
# TEST Use groupByKey() to obtain the counts (2b)
assert sorted(wordCountsGrouped.collect())==[('cat', 2), ('elephant', 1), ('rat', 2)],'incorrect value for wordCountsGrouped'


### (2c) Conteo usando `reduceByKey`

Una mejor solucion es comenzar desde un pair RDD y luego usar la transformacion [reduceByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) para crear un nuevo pair RDD. La transformacion `reduceByKey()` agrupa todas las parejas que comparten la misma clave. Posteriormente aplica la funcion que se le pasa por parametro agrupando los valores de dos en dos. Este proceso se repite iterativamente hasta que obtenemos un unico valor agregado para cada una de las claves del pair RDD. `reduceByKey()` opera aplicando la funcion primero dentro de cada una de las particiones de forma independiente, y posteriormente unicamente comparte los valores agregados entre particiones diferentes, permitiendole escalar de forma eficiente ya que no tiene necesidad de desplazar por la red una gran cantidad de datos.

In [17]:
# TODO: Replace <FILL IN> with appropriate code
# Note that reduceByKey takes in a function that accepts two values and returns a single value
wordCounts = wordPairs.reduceByKey(lambda x,y : x+y)
print (wordCounts.collect())

[('cat', 2), ('elephant', 1), ('rat', 2)]


In [18]:
# TEST Counting using reduceByKey (2c)
assert sorted(wordCounts.collect())==[('cat', 2), ('elephant', 1), ('rat', 2)],'incorrect value for wordCounts'

### (2d) Ahora todo junto

La version mas compleja del codigo ejecuta primero un `map()` sobre el pair RDD, la transformacion `reduceByKey()`, y finalmente la accion `collect()` en una unica linea de codigo.

In [19]:
# TODO: Replace <FILL IN> with appropriate code
wordCountsCollected = (wordsRDD
                       .map(lambda word : (word,1))
                       .reduceByKey(lambda x,y: x+y)
                       .collect())
print(wordCountsCollected)

[('cat', 2), ('elephant', 1), ('rat', 2)]


In [20]:
# TEST All together (2d)
assert sorted(wordCountsCollected)==[('cat', 2), ('elephant', 1), ('rat', 2)],'incorrect value for wordCountsCollected'

## Parte 3: Encontrar las palabras individuales y su frecuencia de aparicion media

### (3a) Palabras unicas

Calcular el numero de palabras unicas en `wordsRDD`. Puedes utitlziar otros RDDs que hayas creado en esta practica si te resulta mas sencillo.

In [182]:
# TODO: Replace <FILL IN> with appropriate code
uniqueWords = wordCounts.count()
print(uniqueWords)

3


In [183]:
# TEST Unique words (3a)
assert uniqueWords== 3, 'incorrect count of uniqueWords'

### (3b) Calular la media usando `reduce()`

Encuentra la frequencia media de aparicion de palabras en `wordCounts`.

Utiliza la accion `reduce()` para sumar los conteos en `wordCounts` y entonces divide por el numero de palabras unicas. Para realizar esto primero aplica un `map()` al pair RDD `wordCounts`, que esta formado por tuplas con el formato (key, value), para convertirlo en un RDD de valores.

In [184]:
# TODO: Replace <FILL IN> with appropriate code
from operator import add
totalCount = (wordCounts
              .map(lambda x: x[1])
              .reduce(add))
average = totalCount / float(wordCounts.count())
print(totalCount)
print(round(average, 2))

5
1.67


In [185]:
# TEST Mean using reduce (3b)
assert round(average, 2)==1.67, 'incorrect value of average'

## Parte 4: Aplicar las funcionalidades desarrolladas a un archivo de texto

Para esto hemos de construir una funcion `wordCount`, capaz de trabajar con datos del mundo real que suelen presentan problemas como el uso de mayusculas o minusculas, puntuacion, acentos, etc. Posteriormente, cargar los datos de nuestra fuente de datos y finalmente, calular el conteo de palabras sobre los datos procesados.

### (4a) funcion `wordCount`

Primero, define una funcion para el conteo de palabras. Deberias reusar las tecnicas que has visto en los apartados anteriores de esta practica. Dicha funcion, ha de tomar un RDD que contenga una lista de palabras, y devolver un pair RDD que contenga todas las palabras con sus correspondientes conteos.

In [186]:
# TODO: Replace <FILL IN> with appropriate code
# TODO: Replace <FILL IN> with appropriate code
def wordCount(wordListRDD):
    """Creates a pair RDD with word counts from an RDD of words.

    Args:
        wordListRDD (RDD of str): An RDD consisting of words.

    Returns:
        RDD of (str, int): An RDD consisting of (word, count) tuples.
    """
    return wordListRDD.map(lambda x : (x,1)).reduceByKey(lambda x,y:x+y)

print(wordCount(wordsRDD).collect())

[('cat', 2), ('elephant', 1), ('rat', 2)]


In [187]:
# TEST wordCount function (4a)
assert sorted(wordCount(wordsRDD).collect())==[('cat', 2), ('elephant', 1), ('rat', 2)],'incorrect definition for wordCount function'

### (4b) Mayusculas y puntuacion

Los ficheros del mundo real son mucho mas complejos que los que hemos estado usando en esta PAC. Algunos de los problemas que son necesarios de solucionar son:
  + Las palabras deben de contarse independientemente de si estan en mayuscula o minuscula (por ejemplo, Spark y spark deberian contarse como la misma palabra).
  + Todos los signos de puntuacion han de eliminarse.
  + Cualquier espacio al principio o al final de la palabra ha de eliminarse.
  
Define la funcion `removePunctuation` que convierta todo el texto a minusculas, elimine los signos de puntuacion, y elimine los espacios al principio y fin de cada palabra. Usa el modulo de Python [re](https://docs.python.org/2/library/re.html) para eliminar cualquier caracter que no sea una letra, un numero o un espacio.

Sino estas familiarizado con las expresiones regulares deberias revisar [este tutorial](https://developers.google.com/edu/python/regular-expressions). Alternativamente, [esta web](https://regex101.com/#python) es de gran ayuda para debugar tus expresiones regulares.

**Hints**

1. Usa la funcion [re.sub()](https://docs.python.org/2.7/library/re.html#re.sub).
2. Para nuestros propositos, "puntuacion" significa "no alphabetico, numerico, o espacio." La expresion regular que define estos caracteres es: `[^A-Za-z\s\d]`
3. No usar `\W`, ya que retendra los guiones bajos.

In [188]:
# TODO: Replace <FILL IN> with appropriate code
import re
def removePunctuation(text):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only whitespace, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        text (str): A string.

    Returns:
        str: The cleaned up string.
    """
    lowertext = text.lower()
    lowertext_rm = re.sub(r'[^a-z0-9\s]','',lowertext)
    lowertext_final = lowertext_rm.strip()
    
    return lowertext_final
    
print(removePunctuation('Hi, you!'))
print(removePunctuation(' No under_score!'))
print(removePunctuation(' *      Remove punctuation then spaces  * '))

hi you
no underscore
remove punctuation then spaces


In [38]:
# TEST Capitalization and punctuation (4b)
assert removePunctuation(" The Elephant's 4 cats. ") == 'the elephants 4 cats', 'incorrect definition for removePunctuation function'

### (4c) Cargar un fichero de texto

Para la siguiente parte, usaremos el texto ya mencionado Lorem Ipsum generado para la pràctica.Para convertir un fichero de texto en un RDD, usaremos el metodo `SparkContext.textFile()`. Tambien usaremos la funcion que acabamos de crear `removePunctuation()` dentro de una transformacion `map()` para eliminar todos los caracteres no alphabeticos, numericos or espacios. Dado que el fichero es bastante grandre, usaremos `take(15)`, de forma que tan solo imprimiremos por pantalla las 15 primeras lineas.

In [29]:
%fs

UsageError: Line magic function `%fs` not found.


In [189]:
# Tan solo ejecuta este codigo
import os.path

fileName = os.path.join('/aula_B2.585/data/', 'LoremIpsum.txt')

loremRDD = sc.textFile(fileName, 8).map(removePunctuation).filter(lambda x: len(x)>0)
loremRDD.take(10)

['aut minima deleniti et autem minus illo esse dolores eligendi corrupti dolore minima nostrum eos nobis nam nihil aspernatur nam ut quae sint laborum ut dolores error possimus aperiam consequatur',
 'pariatur sed quo non itaque qui pariatur saepe ad quis consequatur nihil iste molestias et eos ut expedita vel reiciendis dolorem enim doloribus quam architecto aperiam',
 'sed repudiandae pariatur similique est aut sequi animi in aperiam enim ipsa enim dolorem inventore aut quo odio in consequatur et',
 'aspernatur ad esse et aliquid itaque dolores rerum quia commodi explicabo non magnam nostrum consectetur non sint eum nulla et aut quis doloribus itaque nulla molestiae quis est est quo facilis incidunt a ipsa in itaque sed aut nobis facere dignissimos atque unde cum ea vero',
 'tenetur vel quod voluptatum laudantium dolores neque aut est modi qui aperiam itaque aperiam quae ratione doloremque aut delectus quas qui',
 'qui placeat vel ipsam praesentium sint recusandae dicta minus praesen

### (4d) Extraer las palabras de las lineas

Antes de poder usar la funcion `wordcount()`, hemos de solucionar dos problemas con el formato del RDD:
  + El primer problema es que necesitamos dividir cada linea por sus espacios. ** Esto lo solucionaremos en el apartado (4d). **
  + El segundo problema es que necesitamos filtar las lineas completamente vacias. ** Esto lo solucionaremos en el apartado (4e). **

Para aplicar una transformacion que divida cada elemento del RDD por sus espacios, hemos de aplicar la funcion incorporada en los strings de Python [split()](https://docs.python.org/2/library/string.html#string.split). Cuidado que a primera vista puede parecer que la funcion necesaria es una transformacion `map()`, pero si piensas un poco mas sobre el resultado de la funcion `split()` te daras cuenta que esta no es la opcion correcta.

> Nota:
> * No uses la implementacion estandar del `split()`, pasale un valor de separacion. Por ejemplo, para dividir `line` por comas, usa `line.split(',')`.

In [190]:
# TODO: Replace <FILL IN> with appropriate code
loremWordsRDD = loremRDD.flatMap(lambda x: x.split(" "))
loremWordsCount = loremWordsRDD.count()
print(loremWordsRDD.top(5))
print(loremWordsCount)

['voluptatum', 'voluptatum', 'voluptatum', 'voluptatum', 'voluptatum']
29249699


In [41]:
# TEST Words from lines (4d)
# This test allows for leading spaces to be removed either before or after
# punctuation is removed.
assert loremWordsCount == 29249699, 'incorrect value for loremWordsCount'
assert loremWordsRDD.top(5)==[u'voluptatum', u'voluptatum', u'voluptatum', u'voluptatum', u'voluptatum'], 'incorrect value for loremWordsRDD'

### (4e) Calcula palabras distintas

El siguiente paso es contar cuantas palabras distintas contiene nuestro texto. Puedes usar las transformaciones map() y reduceByKey() ya utilizadas anteriormente.

In [191]:
# TODO: Replace <FILL IN> with appropriate code

distintWordsMapRDD = loremWordsRDD.map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)

distintWordsRDD=distintWordsMapRDD.keys().distinct()

print(distintWordsRDD.take(8))   
print(distintWordsRDD.count())


['tempora', 'sapiente', 'vitae', 'nisi', 'quidem', 'consectetur', 'perferendis', 'debitis']
182


In [192]:
# TEST Remove empty elements (4e)
assert distintWordsRDD.count()== 182, 'incorrect value for shakeWordCount'

In [176]:
print(distintWordsRDD.collect())

['tempora', 'sapiente', 'vitae', 'nisi', 'quidem', 'consectetur', 'perferendis', 'debitis', 'sunt', 'porro', 'ipsum', 'fugit', 'eligendi', 'aspernatur', 'eius', 'fuga', 'placeat', 'consequatur', 'illo', 'earum', 'impedit', 'illum', 'officiis', 'nulla', 'accusantium', 'qui', 'harum', 'pariatur', 'libero', 'reiciendis', 'totam', 'autem', 'occaecati', 'inventore', 'consequuntur', 'ipsam', 'ut', 'repellendus', 'velit', 'id', 'voluptatem', 'corrupti', 'a', 'voluptas', 'itaque', 'nihil', 'animi', 'iure', 'repellat', 'adipisci', 'assumenda', 'temporibus', 'at', 'repudiandae', 'nostrum', 'tenetur', 'atque', 'ratione', 'voluptate', 'cumque', 'reprehenderit', 'sint', 'provident', 'amet', 'iusto', 'vero', 'minus', 'iste', 'quia', 'possimus', 'est', 'quae', 'odio', 'neque', 'accusamus', 'sequi', 'quo', 'mollitia', 'aut', 'excepturi', 'necessitatibus', 'praesentium', 'laboriosam', 'natus', 'minima', 'doloremque', 'delectus', 'aperiam', 'voluptatum', 'eaque', 'doloribus', 'nemo', 'veritatis', 'saepe

### (4f) Cuenta las palabras

Ahora que tenemos un RDD que contiene solo palabras. El siguiente paso es aplicar la funcion `wordCount()` para producir una lista con los conteos de palabras. Podemos ver las 15 mas comunes usando la accion `takeOrdered()`; sin embargo, como los elementos del RRD son pares, necesitamos una funcion especial que ordene los pares de la forma correcta.

Usa las funciones  `wordCount()` y `takeOrdered()` para obtener las 15 palabras mas comunes junto con sus conteos.

In [45]:
# TODO: Replace <FILL IN> with appropriate code
top15WordsAndCounts = wordCount(loremWordsRDD).takeOrdered(15, key = lambda x: -x[1])
print(top15WordsAndCounts)

[('et', 1291567), ('aut', 705322), ('ut', 704966), ('qui', 704703), ('est', 587782), ('voluptatem', 469634), ('quia', 469401), ('rerum', 353054), ('sed', 352873), ('omnis', 352653), ('consequatur', 352436), ('sit', 352396), ('non', 351959), ('voluptas', 351127), ('dolorem', 235822)]


In [47]:
# TEST Count the words (4f)
assert top15WordsAndCounts== [('et', 1291567), ('aut', 705322), 
                              ('ut', 704966), ('qui', 704703), 
                              ('est', 587782), ('voluptatem', 469634), 
                              ('quia', 469401), ('rerum', 353054), ('sed', 352873),
                              ('omnis', 352653), ('consequatur', 352436), 
                              ('sit', 352396), ('non', 351959), ('voluptas', 351127), 
                              ('dolorem', 235822)],'incorrect value for top15WordsAndCounts'

## Parte 5: Calcular algunos estadisticos

Usando las mismas tecnicas que has aplicado en los ejercicios anteriores responde a las siguientes preguntas:

### (5a) ¿Cual es la longitud media de todas las palabras (no repetidas) que aparecen en el texto?

In [181]:
#Calculamos la longitud total de todas las palabras (no repetidas)

len_allwords = distintWordsRDD.map(lambda x : len(x)).reduce(add)

#Teniendo la longitud total lo dividimos para el numero de palabras y obtendremos la longitud media

average_allwords = len_allwords/float(distintWordsRDD.count())

print("La longitud media de las palabras es " + str(round(average_allwords,2)))

La longitud media de las palabras es 6.42


### (5b) ¿Cuantas palabras distintas hay con al menos una 'a' y al menos una 'u' ?

In [180]:
Words_a_u = distintWordsRDD.filter(lambda x: re.match(r'(\b\w*[a]\w*[u]\w*\b)|(\b\w*[u]\w*[a]\w*\b)',x))

print("En el texto seleccionado nos encontramos con {} palabras diferentes que al menos contienen una a y u.".format(Words_a_u.count()))

En el texto seleccionado nos encontramos con 44 palabras diferentes que al menos contienen una a y u.


### (5c) ¿Cuál es la palabra que más se repite y la que menos?

In [167]:
MapReduceWords = loremWordsRDD.map(lambda x: (x,1)).reduceByKey(lambda x,y:x+y)
WordsAndCounts=MapReduceWords.takeOrdered(MapReduceWords.count(),key = lambda x: -x[1])


In [168]:
print("La palabra mas repetida sería '{}' con {} veces".format(WordsAndCounts[0][0],WordsAndCounts[0][1]))
print("La palabra menos repetida sería '{}' con {} veces".format(WordsAndCounts[-1][0],WordsAndCounts[-1][1]))

La palabra mas repetida sería 'et' con 1291567 veces
La palabra menos repetida sería 'esse' con 116586 veces
